In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
offset_chip = [0, 0]

In [15]:
filename = './gRNA/B926/D03658A5.gem'

In [16]:
pattern_x = r"OffsetX=(\d+)"
pattern_y = r"OffsetY=(\d+)"

with open(filename, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match_x = re.search(pattern_x, line)
        match_y = re.search(pattern_y, line)
        if match_x:
            offset_x = int(match_x.group(1))  # 获取匹配到的数值部分
        if match_y:
            offset_y = int(match_y.group(1))
            break

offset_guide = [offset_x, offset_y]
offset_guide

[13, 44]

In [56]:
#offset_guide = [0, 0]

In [17]:
chip_df = pd.read_csv(filename, sep='\t',comment = '#')#, header=None)
chip_df.columns = ['geneID', 'x', 'y', 'MIDCount', 'ExonCount']

In [18]:
chip_df.loc[:,'x'] = chip_df.loc[:, 'x'] + offset_guide[0] - offset_chip[0]
chip_df.loc[:,'y'] = chip_df.loc[:, 'y'] + offset_guide[1] - offset_chip[1]

In [59]:
size = np.argmax([chip_df.loc[:,'y'], chip_df.loc[:,'x']])

In [60]:
bin_size = 100
cnt_bin = int(size / bin_size) + 1
cnt_bin

444

In [61]:
chip_df.loc[:, 'x'] = (chip_df.loc[:, 'x'] / bin_size).astype(int) * bin_size
chip_df.loc[:, 'y'] = (chip_df.loc[:, 'y'] / bin_size).astype(int) * bin_size

In [19]:
merged_df = chip_df.groupby(['geneID', 'x', 'y']).agg({'MIDCount': 'sum', 'ExonCount': 'sum'}).reset_index()
merged_df

,geneID,x,y,MIDCount,ExonCount
0,sgAdrb2_1_gene,845,11357,1,1
1,sgAdrb2_1_gene,875,7435,1,1
2,sgAdrb2_1_gene,883,8869,2,2
3,sgAdrb2_1_gene,918,11349,2,2
4,sgAdrb2_1_gene,931,8916,4,4
...,...,...,...,...,...
390516,sgnon-targeting_2_gene,21140,1387,2,2
390517,sgnon-targeting_2_gene,21267,6203,2,2
390518,sgnon-targeting_2_gene,22813,2005,1,1
390519,sgnon-targeting_2_gene,22957,10569,1,1


In [20]:
merged_df.to_csv(filename + ".offset", sep='\t')

In [63]:
merged_df.to_csv(filename + f".bin_{bin_size}", sep='\t')